In [1]:
# TODO : Normalization
# TODO : add z
# TODO : CTC Loss
from typing import List

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda

#import tensorflow as tf

In [41]:
!jupyter nbconvert lip_keypoint.ipynb --to python

[NbConvertApp] Converting notebook lip_keypoint.ipynb to python
[NbConvertApp] Writing 8638 bytes to lip_keypoint.py


In [78]:
df3 = pd.read_csv("face_landmarks_data3.csv", index_col=0)
df4 = pd.read_csv("face_landmarks_data4.csv", index_col=0)
df5 = pd.read_csv("face_landmarks_data5.csv", index_col=0)
df6 = pd.read_csv("face_landmarks_data6.csv", index_col=0)
df = pd.concat([df3, df4, df5, df6], ignore_index=True)

In [79]:
video_path_counts = df['video_path'].value_counts()

video_paths_to_keep = video_path_counts[video_path_counts == 3000].index

df = df[df['video_path'].isin(video_paths_to_keep)]

In [80]:
df

,frame,x,y,z,visibility,video_path
0,1,0.528295,0.712854,-0.031539,0.0,data3/s3\bbaf1s.mpg
1,1,0.543726,0.710494,-0.030296,0.0,data3/s3\bbaf1s.mpg
2,1,0.559562,0.715571,-0.022568,0.0,data3/s3\bbaf1s.mpg
3,1,0.570321,0.721982,-0.012750,0.0,data3/s3\bbaf1s.mpg
4,1,0.528245,0.728252,-0.018017,0.0,data3/s3\bbaf1s.mpg
...,...,...,...,...,...,...
11989515,75,0.517236,0.742152,-0.018820,0.0,data6/s6\swwv4a.mpg
11989516,75,0.494425,0.741178,-0.009073,0.0,data6/s6\swwv4a.mpg
11989517,75,0.488026,0.755787,-0.010223,0.0,data6/s6\swwv4a.mpg
11989518,75,0.487654,0.740950,-0.002590,0.0,data6/s6\swwv4a.mpg


In [59]:
# mean_x = np.mean(df['x'])
# std_x = np.std(df['x'])
# mean_y = np.mean(df['y'])
# std_y = np.std(df['y'])
# mean_z = np.mean(df['z'])
# std_z = np.std(df['z'])

# # df['x'] = (df['x'] - mean_x) / std_x
# # df['y'] = (df['y'] - mean_y) / std_y
# # df['z'] = (df['z'] - mean_z) / std_z

In [62]:
# df = df[:4000]

In [81]:
df.head(5)

,frame,x,y,z,visibility,video_path
0,1,0.528295,0.712854,-0.031539,0.0,data3/s3\bbaf1s.mpg
1,1,0.543726,0.710494,-0.030296,0.0,data3/s3\bbaf1s.mpg
2,1,0.559562,0.715571,-0.022568,0.0,data3/s3\bbaf1s.mpg
3,1,0.570321,0.721982,-0.012750,0.0,data3/s3\bbaf1s.mpg
4,1,0.528245,0.728252,-0.018017,0.0,data3/s3\bbaf1s.mpg


In [82]:
df.groupby('video_path').count()

,frame,x,y,z,visibility
video_path,,,,,
data3/s3\bbaf1s.mpg,3000,3000,3000,3000,3000
data3/s3\bbaf2p.mpg,3000,3000,3000,3000,3000
data3/s3\bbaf3a.mpg,3000,3000,3000,3000,3000
data3/s3\bbafzn.mpg,3000,3000,3000,3000,3000
data3/s3\bbal4n.mpg,3000,3000,3000,3000,3000
...,...,...,...,...,...
data6/s6\swwpza.mpg,3000,3000,3000,3000,3000
data6/s6\swwv1n.mpg,3000,3000,3000,3000,3000
data6/s6\swwv2s.mpg,3000,3000,3000,3000,3000


In [83]:
df_filtered = df.copy()
chunks = []
num_frames_per_chunk = 3000
for video_path, group in df_filtered.groupby('video_path'):
    num_frames = len(group)
    num_chunks = num_frames // num_frames_per_chunk
    for i in range(num_chunks):
        chunk = group.iloc[i*num_frames_per_chunk:(i+1)*num_frames_per_chunk]
        chunk_reshaped = chunk[['x', 'y', 'z']].values.reshape(-1, 75, 40*3)
        chunks.append(chunk_reshaped)

input_data = np.concatenate(chunks, axis=0)

In [84]:
input_data.shape

(3967, 75, 120)

In [85]:
def create_vocab():
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab


def char_to_int(char):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab.index(char) + 1 if char in vocab else -1

def int_to_char(index):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab[index - 1] if 1 <= index <= len(vocab) else ''

def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens.extend([*line[2]])
            tokens.append(' ')
    return [char_to_int(token) for token in tokens]

all_alignments = []
for video_path in df['video_path'].unique():
    datapath = video_path.split('/')[0]
    speaker_path = video_path.split('/')[-1].split('\\')[0]
    vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]
    
    alignment_path = os.path.join(f'{datapath}','align',f'{vid_path}.align')
    alignments = load_alignments(alignment_path) 
    all_alignments.append(alignments)


In [86]:
len(all_alignments)

3967

In [87]:
# video_path = df['video_path'].iloc[0]
# datapath = video_path.split('/')
# speakert_path = video_path.split('/')[-1].split('\\')[0]
# vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]

In [88]:

label_data = [np.array(label) for label in all_alignments if label]
# Example of shaping your input data and label data for model training
#input_data = np.stack(chunks)  # Ensure that chunks are correctly reshaped and stacked

# Check shapes (important for debugging)
print("Input data shape:", input_data.shape)
print("Number of label sequences:", len(label_data))



Input data shape: (3967, 75, 120)
Number of label sequences: 3967


In [89]:
class LipReadingDataset(Dataset):
    def __init__(self, input_features, labels):
        """
        Args:
            input_features (numpy array): Input features with shape (984, 1, 75, 80)
            labels (numpy array): Labels with shape (984, 30)
        """
        self.input_features = input_features
        self.labels = labels

    def __len__(self):
        return len(self.input_features)

    def __getitem__(self, idx):
        # Convert numpy arrays to torch tensors
        video_frames = torch.from_numpy(self.input_features[idx]).float() 
        character_labels = torch.from_numpy(self.labels[idx]).long()
        return video_frames, character_labels

In [90]:
input_data.shape

(3967, 75, 120)

In [91]:
from torch.utils.data import random_split

def collate_fn(batch):
    # Extract inputs and targets from the batch
    inputs, targets = zip(*batch)

    # Convert inputs and targets into tensors if not already
    inputs_tensor = [torch.tensor(input).float() for input in inputs]
    targets_tensor = [torch.tensor(target).long() for target in targets]

    # Calculate lengths before padding for potential use later (not used in CrossEntropy but useful for other purposes)
    input_lengths = torch.tensor([len(input) for input in inputs_tensor], dtype=torch.long)
    target_lengths = torch.tensor([len(target) for target in targets_tensor], dtype=torch.long)
    max_length = max(max([len(input) for input in inputs_tensor]), 75)
    # Pad inputs and targets
    inputs_padded = pad_sequence(inputs_tensor, batch_first=True, padding_value=0)  # Pad value for inputs usually 0
    targets_padded = pad_sequence([torch.cat([target, torch.tensor([-1] * (max_length - len(target)))] if len(target) < max_length else target) for target in targets_tensor], batch_first=True, padding_value=-1)

    return inputs_padded, targets_padded, input_lengths, target_lengths

dataset = LipReadingDataset(input_data, label_data)

total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)


In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class LipReadingModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_lstm_layers=2, dropout_rate=0.5):
        super(LipReadingModel, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_lstm_layers, batch_first=True, dropout=dropout_rate if num_lstm_layers > 1 else 0)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        x = self.input_layer(x)
        x, _ = self.lstm(x)
        x = self.output_layer(x)
        return x  # Shape: (batch_size, seq_len, num_classes)


vocab = create_vocab()
vocab_len = len(vocab)
num_classes = vocab_len + 1 

In [93]:
vocab

'abcdefghijklmnopqrstuvwxyz123456789 '

In [94]:
vocab_len

36

In [95]:
num_classes

37

In [98]:
import torch
import random
import numpy as np
from torch.utils.data import DataLoader, Subset


def train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=100):
    model.to(device)
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        for inputs, targets, input_lengths, target_lengths in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.transpose(1, 2)  # Shape for Cross-Entropy needs to be (batch_size, num_classes, seq_len)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, targets, input_lengths, target_lengths in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                outputs = outputs.transpose(1, 2)
                val_loss = criterion(outputs, targets)
                total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(test_loader)

        print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = LipReadingModel(input_dim=120, hidden_dim=256, num_classes=num_classes, num_lstm_layers=2, dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Assuming -1 is used for padding in targets
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=500)

C:\Users\User\AppData\Local\Temp\ipykernel_17380\524703463.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs_tensor = [torch.tensor(input).float() for input in inputs]
C:\Users\User\AppData\Local\Temp\ipykernel_17380\524703463.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets_tensor = [torch.tensor(target).long() for target in targets]


Epoch 1, Training Loss: 2.6725718545913697, Validation Loss: 2.448195571899414
Epoch 2, Training Loss: 2.3231738567352296, Validation Loss: 2.2410562229156494
Epoch 3, Training Loss: 2.176622383594513, Validation Loss: 2.1443791294097903
Epoch 4, Training Loss: 2.1147488260269167, Validation Loss: 2.082937469482422
Epoch 5, Training Loss: 2.075247042179108, Validation Loss: 2.0665644931793214
Epoch 6, Training Loss: 2.062563933134079, Validation Loss: 2.055249013900757
Epoch 7, Training Loss: 2.0558971691131593, Validation Loss: 2.053457889556885
Epoch 8, Training Loss: 2.049407982826233, Validation Loss: 2.048344039916992
Epoch 9, Training Loss: 2.0492041373252867, Validation Loss: 2.046921420097351
Epoch 10, Training Loss: 2.0468328261375426, Validation Loss: 2.043192448616028
Epoch 11, Training Loss: 2.0431766176223753, Validation Loss: 2.041975588798523
Epoch 12, Training Loss: 2.0424799406528473, Validation Loss: 2.04042799949646
Epoch 13, Training Loss: 2.042058200836182, Validat

KeyboardInterrupt: 

In [99]:
model.eval()

# No need to track gradients for this operation
with torch.no_grad():
    # Fetch one batch of data
    inputs_padded, targets_padded, input_lengths, target_lengths = next(iter(test_loader))

    # Send this batch to the device (GPU or CPU)
    inputs_padded = inputs_padded.to(device)
    
    # Make predictions
    outputs = model(inputs_padded)
    outputs = outputs.transpose(1, 2)  # Adjust the dimensions for compatibility with Cross-Entropy (batch_size, num_classes, seq_len)

    # Taking the class with maximum probability at each time step
    predicted_indices = outputs.argmax(dim=1)  # Argmax over num_classes
    for i in range(31):
        print("original:", ''.join(int_to_char(idx) for idx in targets_padded[i]))  # Print the original indices
        print("".join([int_to_char(index) for index in predicted_indices[i]]))

C:\Users\User\AppData\Local\Temp\ipykernel_17380\524703463.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs_tensor = [torch.tensor(input).float() for input in inputs]
C:\Users\User\AppData\Local\Temp\ipykernel_17380\524703463.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets_tensor = [torch.tensor(target).long() for target in targets]


original: set blue at t two now 
let redee a    ti no  e  lease  s     ni    ain   ain e se n elease e se e 
original: lay white by e nine now 
laa e iee      f ei  ooain n                                  n n n n      
original: bin green by n seven again 
let redee b  t f  r r sown e n e  e n  a n e se e se e s  e   e se e s se e
original: lay white in k eight again 
lay e ie       ne  noe se a e s  e e n se plse e s  e e sese s se e se e se
original: bin white at sp m eight sp please 
lin reden in y t  eur pn ngean    paon sl n e  e e se e please slsese se e 
original: bin red by m two soon 
lay r dee   by x fe e nge n          e e    e s  eae  n  o      e e se e se
original: lay green in z zero now 
let redee t    h oneo  oo n e n e se e  ase se se e    e   slse eese ee    
original: set red in t six please 
lay eedee    h fi p please s  a e s  e n s     n a e se p s se s se e se e 
original: set red with b six again 
lay e dee    t f e please s  a e   a      e e  e         e a se 